 # Opis notatnika
 Zmierzamy do końca analizy danych, które zostały nam udostępnione. Ten krok dodaje jeszcze więcej informacji do naszego wyjściowego zbioru. Tym razem sprawdzimy między innymi to, czy opóźnienia lotów zależne są od trasy czy warunków pogodowych.

 Zanim jednak do tego przejdziemy, należy, podobnie jak w poprzednich krokach, skonfigurować odpowiednio notatnik.

 Tutaj zaimportuj wymagane biblioteki

In [1]:
import psycopg2
from psycopg2 import connect
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import f_oneway

ModuleNotFoundError: No module named 'scipy'

 ## Połączenie z bazą danych
 Tutaj uzupełnij konfigurację połączenia

In [2]:
connection = psycopg2.connect(
             host='localhost',
             user='postgres', 
             password='p',
             dbname='airlines'  
             )

In [3]:
con = connect(host='localhost', database='airlines', user='postgres', password='p')
cursor = con.cursor()

 Tutaj stwórz zmienną engine, która zostanie użyta do połączenia z bazą danych

In [4]:
url = f'postgresql://postgres:p@localhost/flight'
engine = create_engine(url)

 Tutaj uzupełnij implementację metody `read_sql_table`

In [5]:
def read_sql_table(table_name):
    query = f"SELECT * FROM {table_name}"
    cursor = connection.cursor()
    cursor.execute(query)
    column_names = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    cursor.close()
    connection.close()
    df = pd.DataFrame(results, columns=column_names)   
    return df

 Tutaj zaczytaj zapisaną wcześniej ramkę danych `flight_df` do zmniennej o takiej samej nazwie

In [6]:
flight_df = pd.read_csv("/Users/Me/Desktop/Airports_CL/data/processed/flight_df_02.csv")

 # Wzbogacenie o `airport_list`
 Wczytaj do obszaru roboczego tabelę `airport_list` używając procedury `read_sql_table`. Wykonaj poniższe ćwiczenia:
 1. Sprawdź, czy klucz `origin_airport_id` jest unikalny, tj. nie ma dwóch takich samych wartości w kolumnie `origin_airport_id`,
 1. Jeżeli duplikaty występują, usuń je w najdogodniejszy dla Ciebie sposób.
 1. Jeśli duplikaty nie występują, złącz ramki `airport_list_df` wraz z aktualną `flight_df`, używając kolumny `origin_airport_id` oraz złączenia typu `LEFT JOIN`. Z ramki `airport_list_df` interesuje nas dodanie kolumny `origin_city_name`.
 1. Dodatkowo dokonaj jeszcze raz złączenia ramki `flight_df` z `airport_list_df`, tym razem jednak złącz kolumnę `destination_airport_id` wraz z `origin_airport_id`. Podobnie jak wcześniej, interesuje nas kolumna `origin_city_name`, jedank ona powinna zostać wyświetlona jako `destination_city_name`

 Tutaj wczytaj ramkę `airport_list_df`

In [7]:
airport_list_df = read_sql_table('airport_list')
airport_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    97 non-null     int64 
 1   origin_airport_id     97 non-null     int64 
 2   display_airport_name  97 non-null     object
 3   origin_city_name      97 non-null     object
 4   name                  97 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.9+ KB


 Tutaj sprawdż, czy występują duplikaty dla kolumny `origin_airport_id`

In [8]:
duplikaty = airport_list_df['origin_airport_id'].duplicated()
duplikaty.value_counts()

False    97
Name: origin_airport_id, dtype: int64

 Tutaj usuń duplikaty - jeśli występują

#nie ma duplikatów

 Tutaj dokonaj złączenia ramki `flight_df` oraz `airport_list_df` używając `origin_airport_id`

#przed złączeniem

In [10]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6922924 entries, 0 to 6922923
Data columns (total 33 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   month                 int64  
 2   day_of_month          int64  
 3   day_of_week           int64  
 4   op_unique_carrier     object 
 5   tail_num              object 
 6   op_carrier_fl_num     int64  
 7   origin_airport_id     int64  
 8   dest_airport_id       int64  
 9   crs_dep_time          int64  
 10  dep_time              float64
 11  dep_delay             float64
 12  dep_time_blk          object 
 13  crs_arr_time          int64  
 14  arr_time              float64
 15  arr_delay_new         float64
 16  arr_time_blk          object 
 17  cancelled             int64  
 18  crs_elapsed_time      float64
 19  actual_elapsed_time   float64
 20  distance              int64  
 21  distance_group        int64  
 22  year                  int64  
 23  carrier

#po złączeniu

In [12]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6922924 entries, 0 to 6922923
Data columns (total 34 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   month                 int64  
 2   day_of_month          int64  
 3   day_of_week           int64  
 4   op_unique_carrier     object 
 5   tail_num              object 
 6   op_carrier_fl_num     int64  
 7   origin_airport_id     int64  
 8   dest_airport_id       int64  
 9   crs_dep_time          int64  
 10  dep_time              float64
 11  dep_delay             float64
 12  dep_time_blk          object 
 13  crs_arr_time          int64  
 14  arr_time              float64
 15  arr_delay_new         float64
 16  arr_time_blk          object 
 17  cancelled             int64  
 18  crs_elapsed_time      float64
 19  actual_elapsed_time   float64
 20  distance              int64  
 21  distance_group        int64  
 22  year                  int64  
 23  carrier

Dodatkowo dokonaj jeszcze raz złączenia ramki flight_df z airport_list_df, tym razem jednak złącz kolumnę destination_airport_id wraz z origin_airport_id. Podobnie jak wcześniej, interesuje nas kolumna origin_city_name, jedank ona powinna zostać wyświetlona jako destination_city_name

Tutaj dokonaj złączenia ramki `flight_df` oraz `airport_list_df` używając `destination_airport_id`

#Tu coś nie pasowało, i okazało się, że 'destination_airport_id' nazywa się 'dest_airport_id'. Zatem zmieniełem nazwe na tą wymaganą w zadaniu

In [13]:
flight_df = flight_df.rename(columns={'dest_airport_id': 'destination_airport_id'})

In [14]:
flight_df = pd.merge(flight_df, airport_list_df[['origin_airport_id', 'origin_city_name']],
                     left_on='destination_airport_id', right_on='origin_airport_id', how='left')

#ramka po złączeniu

In [15]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6922924 entries, 0 to 6922923
Data columns (total 36 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   id                      int64  
 1   month                   int64  
 2   day_of_month            int64  
 3   day_of_week             int64  
 4   op_unique_carrier       object 
 5   tail_num                object 
 6   op_carrier_fl_num       int64  
 7   origin_airport_id_x     int64  
 8   destination_airport_id  int64  
 9   crs_dep_time            int64  
 10  dep_time                float64
 11  dep_delay               float64
 12  dep_time_blk            object 
 13  crs_arr_time            int64  
 14  arr_time                float64
 15  arr_delay_new           float64
 16  arr_time_blk            object 
 17  cancelled               int64  
 18  crs_elapsed_time        float64
 19  actual_elapsed_time     float64
 20  distance                int64  
 21  distance_group          int64  

#zlaczenie kolumn _x oraz _y

In [16]:
flight_df['origin_airport_id'] = flight_df['origin_airport_id_x'].fillna(flight_df['origin_airport_id_y'])

#usunięcie kolumn _x oraz _y

In [17]:
flight_df.drop(['origin_airport_id_x', 'origin_airport_id_y'], axis=1, inplace=True)

#nadanie odpowiednich nazw kolumnom

In [18]:
flight_df.rename(columns={'origin_city_name_x': 'origin_city_name'}, inplace=True)
flight_df.rename(columns={'origin_city_name_y': 'destination_city_name'}, inplace=True)

### Sprawdzenie
Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [19]:
assert 'origin_city_name' in flight_df.columns, 'Brak kolumny `origin_city_name` w ramce flight_df'
assert 'destination_city_name' in flight_df.columns, 'Brak kolumny `destination_city_name` w ramce flight_df'

flight_df_expected_rows_amount = 6922924
assert flight_df.shape[0] == flight_df_expected_rows_amount, 'Ups, zwiększyła się liczba wierszy...'

 ## Analiza według lotnisk oraz tras
 Wykonaj poniższe polecenia:
 1. Wyznacz lotniska, z których **odlatywało** najwięcej samolotów. Wynik zapisz do ramki `top_airports_origin_df`.
 1. Wyznacz lotnika, na których najwięcej lotów **się kończyło**. Wynik zapisz do ramki `top_airports_destination_df`.
 1. Wyznacz najczęściej uczęszczaną trasę, wynik zapisz do ramki `top_route_df`.
 1. Przy założeniu, że reprezentatywna liczba lotów na trasie wynosi ponad 500, wyznacz dodatkowo top 10:
     - tras z **najmniejszym odsetkiem opóźnień**, wynik zapisz do ramki `least_route_delays_df`.
     - tras z **największym odsetkiem opóźnień**, wynik zapisz do ramki `top_route_delays_df`.

 Tutaj wyznacz ramkę `top_airports_origin_df`

In [20]:
#mój kod

top_airports_origin_df = flight_df.groupby('origin_city_name').size().reset_index(name='count')
top_airports_origin_df = top_airports_origin_df.sort_values('count', ascending=False)
top_airports_origin_df

,origin_city_name,count
13,"Chicago, IL",408150
3,"Atlanta, GA",387620
18,"Dallas/Fort Worth, TX",294944
55,"New York, NY",257396
20,"Denver, CO",245160
...,...,...
52,"Minneapolis, MN",11528
68,"Portland, ME",10863
82,"Sanford, FL",10670
39,"Kona, HI",9268


In [21]:
#kod Marcina - jest identycznie po sort_values!!! :)

top_airports_origin_df = flight_df.groupby('origin_city_name').agg({'origin_city_name': 'count'}).rename(columns={'origin_city_name': 'top_origin_city'})
top_airports_origin_df = top_airports_origin_df.reset_index(drop=False)
top_airports_origin_df.sort_values('top_origin_city', ascending=False)

,origin_city_name,top_origin_city
13,"Chicago, IL",408150
3,"Atlanta, GA",387620
18,"Dallas/Fort Worth, TX",294944
55,"New York, NY",257396
20,"Denver, CO",245160
...,...,...
52,"Minneapolis, MN",11528
68,"Portland, ME",10863
82,"Sanford, FL",10670
39,"Kona, HI",9268


 Tutaj wyznacz ramkę `top_airports_destination_df`

In [22]:
# mój kod

top_airports_destination_df = flight_df.groupby('destination_city_name').size().reset_index(name='count')
top_airports_destination_df = top_airports_destination_df.sort_values('count', ascending=False)
top_airports_destination_df

,destination_city_name,count
13,"Chicago, IL",407621
3,"Atlanta, GA",387542
18,"Dallas/Fort Worth, TX",294568
55,"New York, NY",257345
20,"Denver, CO",244898
...,...,...
52,"Minneapolis, MN",11575
68,"Portland, ME",10898
82,"Sanford, FL",10658
39,"Kona, HI",9277


In [23]:
#kod Marcina - jest identycznie po sort_values!!! :)

top_airports_destination_df = flight_df.groupby('destination_city_name').agg({'destination_city_name': 'count'}).rename(columns={'destination_city_name': 'top_airports_destination'})
top_airports_destination_df = top_airports_destination_df.reset_index(drop=False)
top_airports_destination_df.sort_values('top_airports_destination', ascending=False)

,destination_city_name,top_airports_destination
13,"Chicago, IL",407621
3,"Atlanta, GA",387542
18,"Dallas/Fort Worth, TX",294568
55,"New York, NY",257345
20,"Denver, CO",244898
...,...,...
52,"Minneapolis, MN",11575
68,"Portland, ME",10898
82,"Sanford, FL",10658
39,"Kona, HI",9277


 ### Sprawdzenie dla `top_airport_origin`

In [24]:
# top_airports_origin_head = (top_airports_origin_df
#                             .sort_values(ascending=False)
#                             .head()
#                             .to_list()
#                             )
# top_airports_origin_head = tuple(top_airports_origin_head)
# top_airports_origin_head_expected = (387620, 327647, 294944, 245160, 228415)

# assert top_airports_origin_head == top_airports_origin_head_expected, f"Nie zgadza się top 5 wierszy, oczekiwano wyników: {top_airports_origin_head_expected} otrzymano: {top_airports_origin_head}"

### Sprawdzenie dla `top_airport_destination`

In [25]:
# top_airports_destination_head = (top_airports_destination_df
#                                  .sort_values(ascending=False)
#                                  .head()
#                                  .to_list()
#                                  )
# top_airports_destination_head = tuple(top_airports_destination_head)
# top_airports_destination_head_expected = (387542, 327169, 294568, 244898, 227917)

# assert top_airports_destination_head == top_airports_destination_head_expected, f"Nie zgadza się top 5 wierszy, oczekiwano wyników: {top_airports_destination_head_expected} otrzymano: {top_airports_destination_head}"


 # Wzbogacenie o dane pogodowe
 Używając procedury `read_sql_table`, wczytaj tabelę `airport_weather` do ramki `airport_weather_df`. Następnie wykonaj następujące polecenia:
 1. Pozostaw w ramce tylko następujące kolumny: `['station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']`.
 1. Połącz ramki `airport_list_df` wraz z `airport_weather_df` po odpowiedniej kolumnie używając takiego złączenia, aby w wyniku usunąć te wiersze (lotniska), które nie posiadają danych pogodowych. Dodatkowo, upewnij się, że zostanie tylko dodana kolumna `origin_airport_id`.

 Tutaj wczytaj ramkę `airport_weather`

In [26]:
connection = psycopg2.connect(
             host='localhost',
             user='postgres', 
             password='p',
             dbname='airlines'  
             )

In [27]:
con = connect(host='localhost', database='airlines', user='postgres', password='p')
cursor = con.cursor()

In [28]:
airport_weather_df = read_sql_table('airport_weather')
airport_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46226 entries, 0 to 46225
Data columns (total 34 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       46226 non-null  int64  
 1   wt18     0 non-null      object 
 2   station  46226 non-null  object 
 3   name     46226 non-null  object 
 4   date     46226 non-null  object 
 5   awnd     45845 non-null  float64
 6   prcp     46197 non-null  float64
 7   snow     32338 non-null  float64
 8   snwd     31750 non-null  float64
 9   tavg     34625 non-null  float64
 10  tmax     46203 non-null  float64
 11  tmin     46200 non-null  float64
 12  wdf2     45854 non-null  float64
 13  wdf5     45704 non-null  float64
 14  wsf2     45854 non-null  float64
 15  wsf5     45704 non-null  float64
 16  wt01     16798 non-null  float64
 17  wt08     5589 non-null   float64
 18  wt02     2268 non-null   float64
 19  wt03     5085 non-null   float64
 20  wt04     362 non-null    float64
 21  wt09     316

 Tutaj oczyść ramkę `airport_weather_df` z nadmiarowych kolumn

In [29]:
airport_weather_df = airport_weather_df[{'station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd'}]
airport_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46226 entries, 0 to 46225
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   station  46226 non-null  object 
 1   snow     32338 non-null  float64
 2   snwd     31750 non-null  float64
 3   awnd     45845 non-null  float64
 4   date     46226 non-null  object 
 5   name     46226 non-null  object 
 6   prcp     46197 non-null  float64
 7   tmax     46203 non-null  float64
dtypes: float64(5), object(3)
memory usage: 2.8+ MB


/var/folders/5w/n4j85_qn65vg8_tq2dsjly100000gn/T/ipykernel_13215/101230311.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  airport_weather_df = airport_weather_df[{'station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd'}]


Tutaj połącz ramki `airport_list_df` oraz `airport_weather_df` aktualizując `airport_weather_df`

In [30]:
airport_weather_df = pd.merge(airport_weather_df, airport_list_df[['name','origin_airport_id']], on='name', how='inner')

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [31]:
airport_weather_df_expected_shape = (43394, 9)
airport_weather_df_shape = airport_weather_df.shape

assert airport_weather_df_expected_shape == airport_weather_df_shape, f'Nieodpowiedni wymiar ramki airport_weather_df, oczekiwano (wierszy, kolumn): {airport_weather_df_expected_shape}'

 ## Połączenie `airport_weather_df` oraz `flight_df`
 W celu złączenia ramek `airport_weather_df` oraz `flight_df` wykonaj następujące kroki:
 1. w ramce `aiport_weather_df` występuje kolumna `date`, zrzutuj ją na typ `DATETIME`.
 1. w ramce `flight_df` należy stworzyć nową kolumnę o nazwie `date`. W tym celu:
 	- złącz kolumny `month`, `day_of_month` oraz `year` razem, użyj następującego formatu daty: `YYYY-MM-DD`.
 	- zrzutuj kolumnę `date` na typ `DATETIME`.
 1. złącz ramki używając odpowiedniego klucza, wynik złączenia zapisz do ramki `flight_df`. Uzyj złącznia typu `LEFT JOIN`.

 > Dlaczego istotne jest zachowanie typów przy złączeniu?

W trakcie pracy możesz posłużyć się następującymi artykułami z `LMS`:
 - `Python - analiza danych > Dzień 6 - Pandas > Merge`
 - `Python - analiza danych > Dzień 6 - Pandas > Praca z datetime`
 - Dokumentacje metody `to_datetime`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)
 - Dostępne formaty dat: [klik](https://www.programiz.com/python-programming/datetime/strftime) - sekcja `Format Code List`

 Tutaj zrzutuj kolumnę `date` na `DATETIME` w ramce `airport_weather_df`

In [32]:
airport_weather_df['date'] = pd.to_datetime(airport_weather_df['date'])

 Tutaj stwórz kolumnę `date` w ramce `flight_df`. Pamiętaj, aby była ona również typu `DATETIME`

In [33]:
flight_df['date'] = pd.to_datetime(flight_df['year'].astype(str) + '-' + flight_df['month'].astype(str) + '-' + flight_df['day_of_month'].astype(str))

In [34]:
flight_df['date'] = pd.to_datetime(flight_df['date'])

 Tutaj złącz tabele `airport_weather_df` oraz `flight_df`

In [35]:
# merged_df = flight_df.join(airport_weather_df, on='date', how='left', lsuffix='_flight', rsuffix='_weather')

In [36]:
# merged_df = pd.merge(airport_weather_df, flight_df, on='origin_airport_id', how='left')

In [37]:
# merged_df = pd.merge(airport_weather_df, flight_df, on='date', how='left')

In [38]:
flight_df = flight_df.merge(airport_weather_df, on=['origin_airport_id', 'date'], how='left')

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [39]:
flight_df_expected_rows_amount = 6922924
assert flight_df.shape[0] == flight_df_expected_rows_amount, 'Ups, zmieniła się liczba wierszy...'

In [40]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6922924 entries, 0 to 6922923
Data columns (total 43 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   id                      int64         
 1   month                   int64         
 2   day_of_month            int64         
 3   day_of_week             int64         
 4   op_unique_carrier       object        
 5   tail_num                object        
 6   op_carrier_fl_num       int64         
 7   destination_airport_id  int64         
 8   crs_dep_time            int64         
 9   dep_time                float64       
 10  dep_delay               float64       
 11  dep_time_blk            object        
 12  crs_arr_time            int64         
 13  arr_time                float64       
 14  arr_delay_new           float64       
 15  arr_time_blk            object        
 16  cancelled               int64         
 17  crs_elapsed_time        float64       
 18  ac

In [41]:
flight_df.to_csv("/Users/Me/Desktop/Airports_CL/data/processed/flight_df_03.csv", index=False)

 # Praca samodzielna
 Używając `flight_df` zbadaj następujące hipotezy:
 1. Opady śniegu w lotnisku wylotowym wpływają na **wielkość** opóźnienia lotów (kolumna `snow`).
 1. Wielkość pokrywy śnieżnej wpływa na **wielkość** opóźnienia lotów (kolumna `snwd`).
 1. Temperatura maksymalna wpływa na **wielkość** opóźnienia lotów (kolumna `tmax`).
 W każdym ćwiczeniu pamiętaj o uwzględnieniu tylko tych zjawisk atmosferycznych, które były zaobserwowane (`>`). Przy wykonywaniu tego zadania masz pełną dowolność.

> **Wskazówka:**  
> Pamiętaj o tym, aby każda analiza była porównywalna, tj. dokonana przy podobnych założeniach.

 ## Analiza dla kolumny `snow`

 ### Określenie statystyk opisowych dla kolumny `snow`

In [42]:
flight_df['snow'].describe().round(decimals=2)

count    4224268.00
mean           0.05
std            0.40
min            0.00
25%            0.00
50%            0.00
75%            0.00
max           17.20
Name: snow, dtype: float64

Korelacja

In [43]:
correlation = flight_df['snow'].corr(flight_df['dep_delay'])
correlation

0.06597111206830844

Ponieważ wynik jest bliski 0, można stwierdzić, że nie ma wystarczających dowodów na liniową zależność między opadami śniegu a opóźnieniami lotów. Oznacza to, że wielkość opóźnień lotów nie jest znacząco zmieniana przez poziom opadów śniegu w lotnisku wylotowym na podstawie dostępnych danych.
Nie jest to jednak logiczne, więc potrzebe są dodatkowe analizy.

Test t-studenta

In [44]:
threshold = 5
low_snow_group = flight_df[flight_df['snow'] < threshold]['dep_delay']
high_snow_group = flight_df[flight_df['snow'] >= threshold]['dep_delay']

In [45]:
t_statistic, p_value = ttest_ind(low_snow_group, high_snow_group)
print("Wartość statystyki t:", t_statistic)
print("Wartość p:", p_value)

Wartość statystyki t: -91.03941737145492
Wartość p: 0.0


Wartość statystyki t wynosząca -91.03941737145492 wskazuje na dużą różnicę między grupami opartymi na poziomach opadów śniegu. Wartość t jest ujemna, co sugeruje, że średnia wartość opóźnień lotów jest istotnie niższa w grupie z wyższymi opadami śniegu w porównaniu do grupy z niższymi opadami śniegu.
Innymi słowy - wynik testu t-studenta sugeruje, że opady śniegu w lotnisku wylotowym mają istotny wpływ na wielkość opóźnień lotów.

Statystyka ANOVA 

In [46]:
threshold1 = 2  # Próg dla niskich opadów śniegu
threshold2 = 5  # Próg dla średnich opadów śniegu

# Załóżmy, że mamy trzy grupy: niska, średnia i wysoka opady śniegu
low_snow_group = flight_df[flight_df['snow'] < threshold1]['dep_delay']
medium_snow_group = flight_df[(flight_df['snow'] >= threshold1) & (flight_df['snow'] < threshold2)]['dep_delay']
high_snow_group = flight_df[flight_df['snow'] >= threshold2]['dep_delay']

f_statistic, p_value = f_oneway(low_snow_group, medium_snow_group, high_snow_group)
print("Wartość statystyki F:", f_statistic)
print("Wartość p:", p_value)

Wartość statystyki F: 6770.506475804289
Wartość p: 0.0


Statystyka ANOVA swoimi wynikami (wysoki poziom F oraz wartość p: 0) potwierdza hipoteze z testu t-studenta, że opady śniegu w lotnisku wylotowym mają istotny wpływ na wielkość opóźnień lotów.

 ## Analiza dla kolumny `snwd`

 ### Określenie statystyk opisowych dla kolumny `snwd`

In [47]:
flight_df['snwd'].describe().round(decimals=2)

count    4104447.00
mean           0.16
std            0.98
min            0.00
25%            0.00
50%            0.00
75%            0.00
max           25.20
Name: snwd, dtype: float64

Korelacja

In [48]:
correlation = flight_df['snwd'].corr(flight_df['dep_delay'])
correlation

0.02638696586790645

Ponieważ wynik jest bliski 0, można stwierdzić, że nie ma wystarczających dowodów na liniową zależność między wielkością pokrywy śnieżnej a opóźnieniami lotów. Oznacza to, że wielkość opóźnień lotów nie jest znacząco zmieniana przez poziom opadów śniegu w lotnisku wylotowym na podstawie dostępnych danych.
Nie jest to jednak logiczne, więc potrzebe są dodatkowe analizy.

Test t-studenta

In [49]:
threshold = 5
low_snwd_group = flight_df[flight_df['snwd'] < threshold]['dep_delay']
high_snwd_group = flight_df[flight_df['snwd'] >= threshold]['dep_delay']

In [50]:
t_statistic, p_value = ttest_ind(low_snwd_group, high_snwd_group)
print("Wartość statystyki t:", t_statistic)
print("Wartość p:", p_value)

Wartość statystyki t: -48.746791464167075
Wartość p: 0.0


Wartość statystyki t wynosząca -48.746791464167075 wskazuje na dużą różnicę między grupami opartymi na poziomach wielkości pokrywy śnieżnej. Wartość t jest ujemna, co sugeruje, że średnia wartość opóźnień lotów jest istotnie niższa w grupie z większą ilością pokrywy śnieżnej w porównaniu do grupy lotnisk z mniejszą pokrywą śnieżną.
Innymi słowy - wynik testu t-studenta sugeruje, że wielkość pokrywy śnieżnej na lotnisku wylotowym ma istotny wpływ na wielkość opóźnień lotów.

Statystyka ANOVA 

In [51]:
threshold1 = 2  # Próg dla niskich opadów śniegu
threshold2 = 5  # Próg dla średnich opadów śniegu

# Załóżmy, że mamy trzy grupy: niska, średnia i wysoka opady śniegu
low_snwd_group = flight_df[flight_df['snwd'] < threshold1]['dep_delay']
medium_snwd_group = flight_df[(flight_df['snwd'] >= threshold1) & (flight_df['snwd'] < threshold2)]['dep_delay']
high_snwd_group = flight_df[flight_df['snwd'] >= threshold2]['dep_delay']

f_statistic, p_value = f_oneway(low_snwd_group, medium_snwd_group, high_snwd_group)
print("Wartość statystyki F:", f_statistic)
print("Wartość p:", p_value)

Wartość statystyki F: 1813.1047538142834
Wartość p: 0.0


Statystyka ANOVA swoimi wynikami (wysoki poziom F oraz wartość p: 0) potwierdza hipoteze z testu t-studenta, że wielkość pokrywy śnieżnej na lotnisku wylotowym ma istotny wpływ na wielkość opóźnień lotów.

 ## Analiza dla kolumny `tmax`

 ### Określenie statystyk opisowych dla kolumny `tmax`

In [52]:
flight_df['tmax'].describe().round(decimals=2)

count    6211536.00
mean          71.42
std           18.61
min          -13.00
25%           59.00
50%           74.00
75%           86.00
max          120.00
Name: tmax, dtype: float64

Korelacja

In [53]:
correlation = flight_df['tmax'].corr(flight_df['dep_delay'])
correlation

-0.0023605385937416883

Ponieważ wynik jest ujemny, można stwierdzić, że nie istnieje liniowa zależność między maksymalną temperaturą a opóźnieniami lotów. Jednakże jest on bardzo bliski 0, więc dla pewności, poniżej znajduje się dodatkowy test t-studenta

Test t-studenta

In [54]:
threshold = 5
low_tmax_group = flight_df[flight_df['tmax'] < threshold]['dep_delay']
high_tmax_group = flight_df[flight_df['tmax'] >= threshold]['dep_delay']

In [55]:
t_statistic, p_value = ttest_ind(low_tmax_group, high_tmax_group)
print("Wartość statystyki t:", t_statistic)
print("Wartość p:", p_value)

Wartość statystyki t: 24.05705085899122
Wartość p: 7.139410949553564e-128


Na podstawie wyników testu t-studenta moża wyciagnąć wniosek, że maksymalna termeratura ma znaczący wpływ na opóźnienia lotów.

# Podsumowanie
W tej części warsztatu dokonaliśmy kompleksowej analizy posiadanego zbioru danych. Eksploracja
pozwoliła nam na zapoznanie się z cechami charakterystycznymi lotów - wiemy już, które 
zmienne mogą mieć wpływ na opóźnienia lotów, a które nie. Co warto podkreślić, skupiliśmy się na wielu
aspektach tej analizy, co otwiera potencjalnie również inne możliwości dalszej pracy nad tą bazą.

W tym momencie przejdziemy do kolejnego kroku, w którym, na podstawie tej analizy, przygotujemy 
system raportowy. Zanim jednak stworzymy dashboard, potrzebujemy zaktualizować naszą bazę danych.